In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
import keras
import keras.activations
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys
from pathlib import Path
import tensorflow as tf
import numpy as np
from sklearn.pipeline import Pipeline
import joblib
# print the current working directory:
p = Path(sys.path[0]).parent.parent.parent.parent


def idxmax(arr):
    _max = 0
    idx = -1
    for i in range(0, len(arr)):
        if arr[i] > _max:
            _max = arr[i]
            idx = i
    return idx

In [ ]:
df_in = pd.read_csv(p / 'second_training_data_detailed.csv', index_col=False)
df_in = df_in.drop(columns=["Unnamed: 0"])
colnames_in = ["currentfret","notename","frequency","maxlevel","overtonefrequency","overtonelevel",0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499]

dict_data = {}

fret_columns = [f"fret {idx}" for idx in range(25)]

for fretname in fret_columns:
    dict_data[fretname] = np.zeros(len(df_in))


for idx, row in df_in.iterrows():
    for fret in range(25):
        score = (25-abs(fret - int(row["currentfret"]))) / 25
        score **= 1.25
        print(fret, row["currentfret"], score)
        dict_data[f"fret {fret}"][idx] = score

df = pd.concat([pd.DataFrame(dict_data), df_in], axis=1)

df.to_csv(p / 'second_training_data_detailed_onehot.csv', index=False)



In [ ]:
df = pd.read_csv(p / 'second_training_data_detailed_onehot.csv', index_col=False).sample(frac=1)

x = df.drop(columns=["notename", "currentfret"] + [f"fret {idx}" for idx in range(25)] + [str(x) for x in range(500)])
y = df[[f"fret {idx}" for idx in range(25)]]

scaler = StandardScaler()
x = scaler.fit_transform(x)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
print(x_train.shape, x_test.shape)


In [ ]:
model = Sequential()


model.add(Dense(4, activation="relu", input_shape=x_train.shape[1:]))
model.add(Dense(32, activation="relu"))
model.add(Dense(25, activation="relu"))


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.005), loss="mean_squared_error", metrics=['accuracy'])


model.fit(x_train, y_train, epochs=150, batch_size=50)

In [ ]:

counted_right = 0
for row, (_, solutions) in zip(x_test, y_test.iterrows()):
    raw_pred = model.predict(row.reshape(-1, 4))
    guessidx = idxmax(list(raw_pred[0]))
    realidx = idxmax(solutions)
    # print(f"guessed: {guessidx} was: {realidx}")
    if abs(guessidx - realidx) < 2:
        counted_right += 1

print(f"accuracy: {counted_right / len(x_test)*100}%")


In [ ]:
model.save('fret_position_model.h5')
joblib.dump(scaler, 'fret_position_scaler.pkl')

In [ ]:
import requests
for row, (_, solutions) in zip(x_test, y_test.iterrows()):
    resp = requests.post("http://localhost:5002/predict", json={"frequency": row[0], "maxlevel": row[1], "overtonefrequency": row[2], "overtonelevel": row[3], "fft_arr": list(row[4:])})
    print(idxmax(solutions), resp.text)

In [ ]:
import shap

# load your data here, e.g. X and y
# create and fit your model here

# load JS visualization code to notebook
shap.initjs()

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_train)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

shap.summary_plot(shap_values, x_train, plot_type="bar")